In [30]:
from scipy.integrate import *
import matplotlib.pyplot as plt
from math import *
import numpy as np
from scipy.optimize import *

In [41]:
class model:
    
    ## Get initial values for the system
    def __init__(self,parameters):
        self.__dict__.update(parameters)
        #if self.nVar != len(X0):
        #    print('ERROR:', self.nVar, 'initial values required')
        #else: 
        #    self.X0 = X0
    
    # SERCA ODE model
    def serca_ode(self,v,t):

        cai, fx1, fx1a, fx2, fy2, fy1a, fy1, cae = v

        dx1 = fx1*(-self.kx1_x1a*cai-self.kx1_y1)+fx1a*self.kx1a_x1+fy1*self.ky1_x1
        dx1a = fx1a*(-self.kx1a_x2*cai-self.kx1a_x1)+fx1*cai*self.kx1_x1a+fx2*self.kx2_x1a
        dx2 = fx2*(-self.kx2_y2-self.kx2_x1a)+fx1a*cai*self.kx1a_x2+fy2*self.ky2_x2

        dy1 = fy1*(-self.ky1_y1a*cae-self.ky1_x1)+fy1a*self.ky1a_y1+fx1*self.kx1_y1
        dy1a = fy1a*(-self.ky1a_y2*cae-self.ky1a_y1)+fy1*cae*self.ky1_y1a+fy2*self.ky2_y1a
        dy2 = fy2*(-self.ky2_x2-self.ky2_y1a)+fy1a*cae*self.ky1a_y2+fx2*self.kx2_y2

        dcae = -cae*(fy1a*self.ky1a_y2 + fy1*self.ky1_y1a) + (fy1a*self.ky1a_y1 + fy2*self.ky2_y1a) + self.kca*(cai - cae)
        
        dcai=0

        return [dcai, dx1, dx1a, dx2, dy2, dy1a, dy1, dcae]
    
    

In [42]:

k_original={
        'kx1_x1a' : 2*1.0e8,
        'kx1a_x2' : 1.0e8,
        'kx1a_x1' : 83.666,
        'kx2_x1a' : 2*83.666,
        'kx2_y2' : 0.6,
        'ky2_x2' : 4.118, #corrected
        'ky2_y1a' : 2*30.015,
        'ky1a_y1' : 30.015,
        'ky1a_y2' : 1.0e5,
        'ky1_y1a' : 2*1.0e5,
        'ky1_x1' : 0.4,
        'kx1_y1' : 1.20e-3,
        'kca' : 0
    }
k=k_original.copy()

In [43]:
def get_SERCA_equilibrium(final_conc):
    cai = 100.0e-9
    my_k=k_original.copy()
    #params for general final conc
    sf=final_conc/250.0
    sm=1
    
    for key in ['kx1_x1a','kx1a_x2']:
        my_k[key]=k_original[key]*sf

    cae=final_conc*1e-6
    #arbitrary inital serca
    v0=[cai]+[9.06424261e-01, 8.66707395e-02, 2.07182701e-03, 3.01868945e-04,
           1.81203090e-03, 2.71927273e-03, cae]
    
    #ODE details
    tstep = 1e-2
    tf = 100
    t = np.linspace(0, tf, tf/tstep+1)

    D=model(my_k)
    # Solve ODE
    sol = odeint(D.serca_ode, v0, t)

    return list(np.array(sol[-1,1:-1]))

In [44]:
#get_SERCA_equilibrium(500)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[1e-07,
 0.6424441123004321,
 0.3071470429086243,
 0.03671109446068794,
 0.005348872466550021,
 0.006421544779175244,
 0.0019273331696295956]

In [121]:
a="""def func():
    global k
    execute_string=[]
    for key in k.keys():
        execute_string.append(key + " = "+str(k[key]))
    print ('; '.join(execute_string))
    exec('; '.join(execute_string))
    print (kx1_x1a,kx1a_x2)"""

In [7]:
#get_SERCA_equilibrium(100)

[0.9064242610635374,
 0.08667073947013512,
 0.0020718270138444204,
 0.0003018689189853863,
 0.0018120308340165466,
 0.002719272784396905]